In [1]:
import numpy as np
from scipy.linalg import expm, logm
import sys
import copy
import time
import cv2
import os
from PIL import Image

In [2]:
################ Pre-defined parameters and functions below (can change if needed) ################
PI = np.pi
# 20Hz
SPIN_RATE = 20  

# UR3 home location
home = [270*PI/180.0, -90*PI/180.0, 90*PI/180.0, -90*PI/180.0, -90*PI/180.0, 135*PI/180.0]  

# UR3 current position, using home position for initialization
current_position = copy.deepcopy(home)  

thetas = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

digital_in_0 = 0
analog_in_0 = 0.0

suction_on = True
suction_off = False

current_io_0 = False
current_position_set = False

In [3]:
from matplotlib.pyplot import imshow
from matplotlib.pyplot import cm



# Create directory if it doesn't exist
if not os.path.exists("threshold_images"):
    os.makedirs("threshold_images")

In [4]:
def IMG2W(row, col, image):
    """Transform image coordinates to world coordinates

    Parameters
    ----------
    row : int
        Pixel row position
    col : int
        Pixel column position
    image : np.ndarray
        The given image (before or after preprocessing)

    Returns
    -------
    x : float
        x position in the world frame
    y : float
        y position in the world frame
    """
    x, y = 0.0, 0.0
    return x, y

In [5]:
def draw_image(world_keypoints):
    """Draw the image based on detecte keypoints in world coordinates

    Parameters
    ----------
    world_keypoints:
        a list of keypoints detected in world coordinates
    """
    pass


"""
Program run from here
"""

'\nProgram run from here\n'

In [358]:
def find_centerline(edges):
    """Find centerline between Canny edges and return as list of points"""
    # Fill the space between edges
    filled = edges.copy()
    h, w = edges.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(filled, mask, (0,0), 255)
    
    # Invert to get the shapes
    filled_inv = cv2.bitwise_not(filled)
    
    # Get distance transform
    dist = cv2.distanceTransform(filled_inv, cv2.DIST_L2, 5)
    
    # Convert to 8-bit for thresholding
    dist_8u = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    
    # Threshold to get centerline
    _, centerline = cv2.threshold(dist_8u, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours of the centerline
    contours, _ = cv2.findContours(centerline, cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_KCOS)
    
    # Convert to list of points
    centerline_points = []
    spacing = 5  # Adjust spacing between points as needed
    
    for contour in contours:
        # Optional: approximate contour to reduce points
        epsilon = 0.005 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, False)
        
        # Sample points with spacing
        for i in range(0, len(approx), spacing):
            point = tuple(contour[i][0])
            centerline_points.append(point)
    
    return centerline_points

In [527]:
def find_keypoints2(image):
    """Gets keypoints along center of lines with reduced density
    
    Parameters
    ----------
    image : np.ndarray
        The given image (before or after preprocessing)
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # Threshold to ensure clean binary image
    # _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                             cv2.THRESH_BINARY, 3, -10)
    # Fill in the lines to get solid regions
    kernel = np.ones((3,3), np.uint8)
    filled = cv2.dilate(binary, kernel, iterations=1)
    
    # Get the distance transform
    dist = cv2.distanceTransform(filled, cv2.DIST_L2, 5)
    
    # Threshold the distance transform to get center line
    _, center = cv2.threshold(dist, 1, 255, cv2.THRESH_BINARY)
    center = center.astype(np.uint8)
    canny = cv2.Canny(center,1,255, L2gradient=True)
    # Find contours of the center line
    contours, _ = cv2.findContours(center, cv2.RETR_LIST, 
                                 cv2.CHAIN_APPROX_NONE)
    height, width = center.shape[:2]
    
    # Process contours to get keypoints
    keypoints = []
    min_length = 0  # Increased minimum length
    spacing = 15  # Increased spacing between points
    for contour in contours:
        if len(contour) < min_length:
            continue
        epsilon = 0.00009 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)        
        for i in range(0, len(approx), spacing):
            # point = approx[i][0]
            point = contour[i][0]
            # keypoints.append(tuple(point))
            keypoints.append(tuple(point))
            
    # Create debug images
    height, width = image.shape[:2]
    recreated_image = np.ones((height, width, 3), dtype=np.uint8) * 255  # White background
    # Draw lines connecting keypoints
    if len(keypoints) > 1:
        for i in range(0, len(keypoints) - 1):  # Changed to -1 to avoid going past end
            start_point = keypoints[i]
            end_point = keypoints[i + 1]
            cv2.line(recreated_image, start_point, end_point, (0, 0, 255), 1)
    debug_image = image.copy()
    if len(debug_image.shape) == 2:
        debug_image = cv2.cvtColor(debug_image, cv2.COLOR_GRAY2BGR)
    
    # Draw keypoints
    keypoint_image = debug_image.copy()
    for x, y in keypoints:
        cv2.circle(keypoint_image, (x, y), 3, (0, 0, 255), -1)
        
    # Save debug images
    cv2.imwrite('2debug_6_cany.png',canny)
    cv2.imwrite('2debug_5_draw.png', recreated_image)
    cv2.imwrite('2debug_1_binary.png', binary)
    cv2.imwrite('2debug_2_filled.png', filled)
    cv2.imwrite('2debug_3_center.png', center)
    cv2.imwrite('2debug_4_keypoints.png', keypoint_image)
    return keypoints

In [528]:
zigzag = cv2.imread("images/zigzag.jpg")
turkey = cv2.imread("images/turkey.png")
status = cv2.imread("images/status.png")


In [529]:
def find_keypoints(image):
    """Gets keypoints from the given image

    Parameters
    ----------
    image : np.ndarray
        The given image (before or after preprocessing)

    Returns
    -------
    keypoints
        a list of keypoints detected in image coordinates
  `  """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Threshold to get black lines
    # _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    _,binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                             cv2.THRESH_BINARY, 5, -1000)
    # Create a copy for visualization
    # Get distance transform
    kernel = np.ones((5,5), np.uint8)
    filled = cv2.dilate(binary, kernel, iterations=2)
    dist = cv2.distanceTransform(filled, cv2.DIST_L2, 3)
    # Threshold distance transform to get ridges (centerlines)
    _, ridges = cv2.threshold(dist, 1, 255, cv2.THRESH_BINARY)
    ridges = ridges.astype(np.uint8)
    
    # Find contours of the ridges
    contours, hierarchy = cv2.findContours(ridges, cv2.RETR_LIST , cv2.CHAIN_APPROX_NONE )
    keypoints = []
    min_contour_length = 0
    for contour_num in range(len(contours)):
        if len(contours[contour_num]) < min_contour_length:
            continue
        # Approximate the contour to get main points
        epsilon = 0.001 * cv2.arcLength(contours[contour_num], True)
        approx = cv2.approxPolyDP(contours[contour_num], epsilon, False)
        
        # Get points with consistent spacing
        for i in range(len(approx) - 1):
            p1 = tuple(approx[i][0])
            p2 = tuple(approx[i + 1][0])
            
            # Calculate distance between points
            dx = p2[0] - p1[0]
            dy = p2[1] - p1[1]
            dist = np.sqrt(dx*dx + dy*dy)
            
            # Add points with fixed spacing
            if dist > 20:
                steps = int(dist / 20)
                for j in range(steps + 1):
                    t = j / steps
                    x = int(p1[0] + dx * t)
                    y = int(p1[1] + dy * t)
                    keypoints.append((x, y))
            else:
                keypoints.append(p1)
                
        # Add last point
        if len(approx) > 0:
            keypoints.append(tuple(approx[-1][0]))
    
    # Remove duplicates while maintaining order
    keypoints = list(dict.fromkeys(keypoints))
    
    # Draw debug visualizations
    debug_image = image.copy()
    if len(debug_image.shape) == 2:
        debug_image = cv2.cvtColor(debug_image, cv2.COLOR_GRAY2BGR)
    
    # Draw ridges
    ridge_image = debug_image.copy()
    cv2.drawContours(ridge_image, contours, -1, (0, 255, 0), 1)
    
    # Draw keypoints and connecting lines
    keypoint_image = debug_image.copy()
    
    # # Draw lines between consecutive points
    # for i in range(len(keypoints) - 1):
    #     pt1 = keypoints[i]
    #     pt2 = keypoints[i + 1]
    #     cv2.line(keypoint_image, pt1, pt2, (0, 255, 0), 1)
    
    # Draw keypoints
    for x, y in keypoints:
        cv2.circle(keypoint_image, (x, y), 2, (0, 0, 255), -1)
    
    # Save debug images
    cv2.imwrite('debug_1_binary.png', binary)
    cv2.imwrite('debug_2_ridges.png', ridges)
    cv2.imwrite('debug_3_centerlines.png', ridge_image)
    cv2.imwrite('debug_4_keypoints.png', keypoint_image)

    
    return keypoints

In [530]:
# find_keypoints(status)
# find_keypoints(zigzag)
key = find_keypoints2(turkey)

In [531]:
# def find_keypoints(image):
#     """Gets keypoints along the centerlines of the black lines in the drawing
    
#     Parameters
#     ----------
#     image : np.ndarray
#         The given image (before or after preprocessing)
#     """
#     # Convert to grayscale if needed
#     if len(image.shape) == 3:
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray = image.copy()
    
#     # Threshold to get black lines
#     _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    
#     # Create a copy for visualization
#     vis_image = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    
#     # Get distance transform
#     dist = cv2.distanceTransform(binary, cv2.DIST_L2, 5)
    
#     # Threshold distance transform to get ridges (centerlines)
#     _, ridges = cv2.threshold(dist, 1, 255, cv2.THRESH_BINARY)
#     ridges = ridges.astype(np.uint8)
    
#     # Find contours of the ridges
#     contours, _ = cv2.findContours(ridges, cv2.RETR_LIST, 
#                                  cv2.CHAIN_APPROX_NONE)
    
#     keypoints = []
#     min_contour_length = 30
    
#     for contour in contours:
#         if len(contour) < min_contour_length:
#             continue
            
#         # Approximate the contour to get main points
#         epsilon = 0.01 * cv2.arcLength(contour, True)
#         approx = cv2.approxPolyDP(contour, epsilon, False)
        
#         # Get points with consistent spacing
#         for i in range(len(approx) - 1):
#             p1 = tuple(approx[i][0])
#             p2 = tuple(approx[i + 1][0])
            
#             # Calculate distance between points
#             dx = p2[0] - p1[0]
#             dy = p2[1] - p1[1]
#             dist = np.sqrt(dx*dx + dy*dy)
            
#             # Add points with fixed spacing
#             if dist > 20:
#                 steps = int(dist / 20)
#                 for j in range(steps + 1):
#                     t = j / steps
#                     x = int(p1[0] + dx * t)
#                     y = int(p1[1] + dy * t)
#                     keypoints.append((x, y))
#             else:
#                 keypoints.append(p1)
                
#         # Add last point
#         if len(approx) > 0:
#             keypoints.append(tuple(approx[-1][0]))
    
#     # Remove duplicates while maintaining order
#     keypoints = list(dict.fromkeys(keypoints))
    
#     # Draw debug visualizations
#     debug_image = image.copy()
#     if len(debug_image.shape) == 2:
#         debug_image = cv2.cvtColor(debug_image, cv2.COLOR_GRAY2BGR)
    
#     # Draw ridges
#     ridge_image = debug_image.copy()
#     cv2.drawContours(ridge_image, contours, -1, (0, 255, 0), 1)
    
#     # Draw keypoints and connecting lines
#     keypoint_image = debug_image.copy()
    
#     # Draw lines between consecutive points
#     for i in range(len(keypoints) - 1):
#         pt1 = keypoints[i]
#         pt2 = keypoints[i + 1]
#         cv2.line(keypoint_image, pt1, pt2, (0, 255, 0), 1)
    
#     # Draw keypoints
#     for x, y in keypoints:
#         cv2.circle(keypoint_image, (x, y), 2, (0, 0, 255), -1)
    
#     # Save debug images
#     cv2.imwrite('debug_1_binary.png', binary)
#     cv2.imwrite('debug_2_ridges.png', ridges)
#     cv2.imwrite('debug_3_centerlines.png', ridge_image)
#     cv2.imwrite('debug_4_keypoints.png', keypoint_image)
    
#     return keypoints